In [2]:
import numpy as np
# import cv2 
import os
from scipy import sparse
from sklearn.preprocessing import normalize
import scipy
import tqdm

In [5]:
lumask=np.array([[1,0,0],[0,1,0],[0,0,0]])
np.where(lumask!=0)[0]

array([0, 1])

## 1.数据集导入

In [2]:
class Dataset():
    def __init__(self,root="./DiLiGenT/pmsData/ballPNG"):
        self.img=[]
        self.directions=[]
        self.intensities=[]
        dir_direction=os.path.join(root,"light_directions.txt")
        dir_intensity=os.path.join(root,"light_intensities.txt")
        #mask指出物体在图片中区域
        dir_mask=os.path.join(root,"mask.png")
        file_list=os.path.join(root,"filenames.txt")
        self.directions=np.array(self.load_txt(dir=dir_direction))
        self.intensities=np.array(self.load_txt(dir=dir_intensity))
        self.load_img(root=root,dir=file_list)
        self.H,self.W,_=self.img[0].shape
        self.load_mask(filename=dir_mask)
    def load_txt(self,dir=""):
        txt_list=[]
        with open(dir) as f:
            txt_list=f.read().splitlines()
            for i in range(len(txt_list)):
                txt_list[i]=txt_list[i].split()
                txt_list[i]=[float(x) for x in txt_list[i]]
        return txt_list
    def load_img(self,root="",dir=""):
        with open(dir) as f:
            img_dirs=f.read().splitlines()
        for img_dir in img_dirs:
            img_path=os.path.join(root,img_dir)
            img=cv2.imread(img_path)
            self.img.append(img)
    def load_mask(self, filename=None):
        self.mask=cv2.imread(filename,0)
        mask=self.mask.reshape((-1,1))
        self.foreground_ind=np.where(mask != 0)[0]
        self.background_ind=np.where(mask == 0)[0]
    def normal_imags(self):
        for i in tqdm.tqdm(range(len(self.intensities))):
            intensity=self.intensities[i]
            
            img=self.img[i]
            h,w,_=img.shape
            for j in range(h):
                for k in range(w):
                   
                    img[j,k]=img[j,k]/intensity

## 2.法向量map计算

In [3]:
def get_normal(img,directions,background_ind):
    print(np.array(img).shape)
    imgs=[np.mean(x,axis=2) for x in img]
    print(np.array(imgs).shape)
    img=None
    H, W = imgs[0].shape
    for i in tqdm.tqdm(range(len(imgs))):
        if img is None:
            img = imgs[i].reshape((-1, 1))
        else:
            img = np.append(img, imgs[i].reshape((-1, 1)), axis=1)
    print(img.shape)
    N = scipy.linalg.lstsq(directions, img.T)[0].T
    N = normalize(N, axis=1)  
    #让所有的非物体上的法向量为0
    if background_ind is not None:
        for i in range(N.shape[1]):
            N[background_ind, i] = 0
        
    #应该便于索引
    N = np.reshape(N, (H, W, 3))
    return N

## 3.根据法向量恢复深度

In [5]:
def compute_depth(N,mask):
    H,W= mask.shape
    # 得到掩膜图像非零值索引（即物体区域的索引）
    obj_h, obj_w = np.where(mask != 0)
    # 得到非零元素的数量
    no_pix = np.size(obj_h)
    # 构建一个矩阵 里面的元素值是掩膜图像索引的值
    index = np.zeros((H, W))
    for idx in range(no_pix):
        index[obj_h[idx], obj_w[idx]] = idx
        
    #稀疏矩阵
    M = sparse.lil_matrix((2*no_pix, no_pix))
    v = np.zeros((2*no_pix, 1))

    for idx in tqdm.tqdm(range(no_pix)):
        h = obj_h[idx]
        w = obj_w[idx]
        n_x = N[h,w,0]
        n_y = N[h,w,1]
        n_z = N[h,w,2]+1e-8
        #分情况讨论，看看像素点是否在边界上或者物体内或者物体外
        if index[h,w+1] and index[h-1,w]:
            M[2*idx, index[h,w]]=(n_z+1e-8)
            M[2*idx, index[h,w+1]]=-(n_z+1e-8)
            v[2*idx,0]=n_x

            M[2*idx+1, index[h,w]]=(n_z+1e-8)
            M[2*idx+1, index[h-1,w]]=-(n_z+1e-8)
            v[2*idx+1,0]=n_y
        #在物体边界，看看另一个方向上的点在不在物体内，在，则用另一个点算，否则为0
        elif index[h-1,w]:
            f = -1
            if index[h, w+f]:
                M[2*idx, index[h, w]] = (n_z+1e-8)
                M[2*idx, index[h, w+f]]= -(n_z+1e-8)
                v[2*idx, 0] = f * n_x 
            M[2*idx+1, index[h, w]] = (n_z+1e-8)
            M[2*idx+1, index[h-1, w]]= -(n_z+1e-8)
            v[2*idx+1, 0] = n_y 
            
        elif index[h, w+1]:
            f = -1
            if index[h-f, w]:
                M[2*idx, index[h, w]] = (n_z+1e-8)
                M[2*idx, index[h-f, w]]= -(n_z+1e-8)
                v[2*idx, 0] = f * n_y 
            M[2*idx+1, index[h, w]] = (n_z+1e-8)
            M[2*idx+1, index[h, w+1]]= -(n_z+1e-8)
            v[2*idx+1, 0] = n_x 
        #都是0，看看另一个方向上点有没有在物体内的，在就把式子写进去。
        else:
            f = -1
            if index[h, w+f]:
                M[2*idx, index[h, w]] = (n_z+1e-8)
                M[2*idx, index[h, w+f]]= -(n_z+1e-8)
                v[2*idx, 0] = f * n_x 
            if index[h-f, w]:
                M[2*idx+1, index[h, w]] = (n_z+1e-8)
                M[2*idx+1, index[h-f, w]]= -(n_z+1e-8)
                v[2*idx+1, 0] = f * n_y 
    #填完矩阵就解方程。
    A=M.T.dot(M)
    B=M.T.dot(v)
    z=sparse.linalg.spsolve(A,B)
    #z最小的变成1
    z = z - min(z)
    depth=np.zeros((H,W))
    for idx in range(no_pix):
        # 2D图像中的位置
        h = obj_h[idx]
        w = obj_w[idx]
        depth[h, w] = z[idx]
    return depth

## 4.保存深度图

In [10]:
def save_depth(depth,save_dir=""):
    if save_dir is "":
        raise Exception("FilePathNULL")
    h,w=depth.shape
    f = open(save_dir, 'w')
    for i in tqdm.tqdm(range(h)):
        for j in range(w):
            if depth[i, j] > 0:
                seq = 'v' + ' ' + str(float(i)) + ' ' + str(float(j)) + ' ' + str(depth[i, j]) + '\n'
                f.writelines(seq)

<>:2: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:2: SyntaxWarning: "is" with a literal. Did you mean "=="?
/var/folders/wz/_wr6pzwx5y55_s2xfc7dzfhm0000gn/T/ipykernel_55605/3999285899.py:2: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if save_dir is "":


## 5.主函数

In [9]:
root="./DiLiGenT/pmsData/"
file_list=os.listdir("./DiLiGenT/pmsData/")
for i in range(len(file_list)):
    if file_list[i][-3:]!= "PNG":
        continue
    else:
        print("Start",file_list[i])
    if not os.path.exists("./result/"+file_list[i]):
        os.mkdir("./result/"+file_list[i])
    img_dir=os.path.join(root,file_list[i])
    save_dir="./result/"+file_list[i]+"/depth"+".obj"
    dataset=Dataset(root=img_dir)
    
    print("-----图像归一化-----")
    dataset.normal_imags()
    print("-----求解法向量图-----")
    N=get_normal(dataset.img,dataset.directions,dataset.background_ind)
    print("-----恢复深度-----")
    depth=compute_depth(N,dataset.mask)
    save_depth(depth,save_dir)
    
    #保存法向量图
    N[:,:,0], N[:,:,2] = N[:,:,2], N[:,:,0].copy()
    N = (N + 1.0) / 2.0
    cv2.imwrite("./result/"+file_list[i]+"/normal.png",N*255)

Start bearPNG
-----normal images-----


100%|███████████████████████████████████████████| 96/96 [00:21<00:00,  4.53it/s]


-----compute normal-----
(96, 512, 612, 3)
(96, 512, 612)


100%|███████████████████████████████████████████| 96/96 [00:02<00:00, 42.99it/s]


(313344, 96)
-----comput depth-----


...

100%|███████████████████████████████████████████| 96/96 [00:21<00:00,  4.52it/s]


-----compute normal-----
(96, 512, 612, 3)
(96, 512, 612)


100%|███████████████████████████████████████████| 96/96 [00:02<00:00, 47.24it/s]


(313344, 96)
-----comput depth-----


100%|███████████████████████████████████████| 512/512 [00:00<00:00, 3324.45it/s]
